In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/link-da/2.-Harry-Potter-and-the-Chamber-of-Secrets.pdf
/kaggle/input/link-da/4.-Harry-Potter-and-the-Goblet.pdf
/kaggle/input/link-da/1.-Harry-Potter-and-the-Philosophers-Stone.pdf
/kaggle/input/link-da/3.-Harry-Potter-and-the-Prisoner-of-Azkaban.pdf


In [2]:
pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.5 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip install --upgrade scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 86.7 MB/s eta 0:00:00:00:01:01
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 0.22.0 requires google-cloud-bigquery[bqstorage,pandas]>=3.10.0, but you have google-cloud-bigquery 2.34.4 which is incompatible.
bigframes 0.22.0 requires google-cloud-storage>=2.0.0, but you have google-cloud-storage 1.44.0 which is incompatible.
bigframes 0.22.0 requires pandas<2.1.4,>=1.5.0, but you have pandas 2.2.2 which is incompatible.
dataproc-jupyter-plugin 0.1.79 requires pydantic~=1.10.0, but you have pydantic 2.8.2 which is incompatible.
spaghetti 1.7.6 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.

In [4]:
import sklearn
print(sklearn.__version__)

1.5.2


# Task 1b: Implement code to build the Term-document Matrix

In [5]:
import PyPDF2

#Extracting
def extract_text_from_pdf(pdf_file):
    text = ""
    with open(pdf_file, 'rb') as file:
        reader = PyPDF2.PdfReader(file)  # Use PdfReader instead of PdfFileReader
        for page_num in range(len(reader.pages)):  # reader.pages gives a list of pages
            page = reader.pages[page_num]
            text += page.extract_text()
    return text

book1 = extract_text_from_pdf("/kaggle/input/link-da/1.-Harry-Potter-and-the-Philosophers-Stone.pdf")
book2 = extract_text_from_pdf("/kaggle/input/link-da/2.-Harry-Potter-and-the-Chamber-of-Secrets.pdf")
book3 = extract_text_from_pdf("/kaggle/input/link-da/3.-Harry-Potter-and-the-Prisoner-of-Azkaban.pdf")
book4 = extract_text_from_pdf("/kaggle/input/link-da/4.-Harry-Potter-and-the-Goblet.pdf")

In [6]:
# Text Preprocessing:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

nltk.download('punkt')
nltk.download('stopwords')

def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word.isalnum() and word not in stop_words]
    return tokens

processed_text1 = preprocess_text(book1)
processed_text2 = preprocess_text(book2)
processed_text3 = preprocess_text(book3)
processed_text4 = preprocess_text(book4)


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
import time
from sklearn.feature_extraction.text import CountVectorizer

def ensure_string(text):
    if isinstance(text, list):
        return ' '.join(text)  

def build_term_document_matrix(documents):
    vectorizer = CountVectorizer() 
    X = vectorizer.fit_transform(documents)  
    return X, vectorizer.get_feature_names_out()  

documents = [
    ensure_string(processed_text1),
    ensure_string(processed_text2),
    ensure_string(processed_text3),
    ensure_string(processed_text4)
]

start_time = time.time()

term_document_matrix, terms = build_term_document_matrix(documents)

execution_time = time.time() - start_time


print("Term-Document Matrix (Raw Frequency):")
print(term_document_matrix.toarray()) 

print("\nTerms (Features):")
print(terms)  

print(f"\nExecution Time: {execution_time} seconds")

Term-Document Matrix (Raw Frequency):
[[ 1  1  1 ...  2  0  1]
 [ 1  1  1 ...  0  0  0]
 [ 1  1  1 ...  3  0  0]
 [ 1  1  1 ... 11  1  3]]

Terms (Features):
['10' '100' '101' ... 'zooming' 'éclair' 'éclairs']

Execution Time: 0.26993680000305176 seconds


# Task 1c: Compute Similarities Using Euclidean Distance and Cosine Similarity

In [8]:
# Euclidean Distance
from sklearn.metrics.pairwise import euclidean_distances

euclidean_dist = euclidean_distances(term_document_matrix)
print("Euclidean Distance Matrix:\n", euclidean_dist)


Euclidean Distance Matrix:
 [[   0.          916.1129843  1463.21256146 3420.60564813]
 [ 916.1129843     0.         1134.279507   2952.26082859]
 [1463.21256146 1134.279507      0.         2456.55042692]
 [3420.60564813 2952.26082859 2456.55042692    0.        ]]


Euclidean distance shows how far apart the books are in the vector space. Lower values indicate more similarity.

In [9]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim = cosine_similarity(term_document_matrix)
print("Cosine Similarity Matrix:\n", cosine_sim)


Cosine Similarity Matrix:
 [[1.         0.94329679 0.9416178  0.93467331]
 [0.94329679 1.         0.95078986 0.95214857]
 [0.9416178  0.95078986 1.         0.94577085]
 [0.93467331 0.95214857 0.94577085 1.        ]]


Cosine similarity will indicate how similar the books are based on the angle between their term vectors. Values closer to 1 indicate more similarity.

# Task 1d: Select representative words and compute reduced embedding.

In [10]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD

documents = [' '.join(processed_text1), 
             ' '.join(processed_text2), 
             ' '.join(processed_text3), 
             ' '.join(processed_text4)]

representative_words = ['wizard', 'magical', 'harry', 'spells', 'witch', 'journey']

# Create a CountVectorizer with the selected words
vectorizer = CountVectorizer(vocabulary = representative_words)
term_document_matrix = vectorizer.fit_transform(documents).toarray()

df = pd.DataFrame(term_document_matrix, columns = representative_words)
print("Term-Document Matrix for Selected Words:\n", df)

Term-Document Matrix for Selected Words:
    wizard  magical  harry  spells  witch  journey
0      42       11   1308       6     12       11
1      47       16   1633       9     16        5
2      39       31   2035       2     42        6
3      83      125   3134      24     37       10


In [11]:
# Compute cosine similarity
cosine_sim = cosine_similarity(term_document_matrix)
print("Cosine Similarity Matrix for Selected Words:\n", cosine_sim)

Cosine Similarity Matrix for Selected Words:
 [[1.         0.99997861 0.99980603 0.99946822]
 [0.99997861 1.         0.9998701  0.99954149]
 [0.99980603 0.9998701  1.         0.999609  ]
 [0.99946822 0.99954149 0.999609   1.        ]]


By focusing on key words related to the Harry Potter universe, the similarity scores will increase, as the key words will represent the core themes of each book more accurately than the full vocabulary.

In [12]:
# Reduce dimensionality using Truncated SVD
n_components = 2
svd = TruncatedSVD(n_components=n_components)
reduced_embeddings = svd.fit_transform(term_document_matrix)

print("Reduced Embeddings:\n", reduced_embeddings)

Reduced Embeddings:
 [[1308.55565611  -23.64973942]
 [1633.60579063  -26.99254366]
 [2035.79150708  -26.59805943]
 [3137.64510112   41.17428047]]


# Task 7: Derive the formula for Conditional Entropy

## Entropy

The entropy \( H(X) \) of a discrete random variable \( X \) with probability mass function \( p(x) \) is defined as:

$$
H(X) = - \sum_{x} p(x) \log p(x)
$$

This measures the average amount of uncertainty or information contained in the random variable \( X \).

## Joint Entropy

The joint entropy \( H(X, Y) \) of two discrete random variables \( X \) and \( Y \) with joint probability mass function \( p(x, y) \) is defined as:

$$
H(X, Y) = - \sum_{x, y} p(x, y) \log p(x, y)
$$

This measures the uncertainty or information contained in the pair \( (X, Y) \).

## Conditional Probability

The conditional probability \( p(x|y) \) of \( X \) given \( Y \) is defined as:

$$
p(x|y) = \frac{p(x, y)}{p(y)}
$$

This represents the probability of \( X \) occurring given that \( Y \) has occurred.

## Conditional Entropy

The conditional entropy \( H(X|Y) \) quantifies the amount of information needed to describe the outcome of \( X \) given that the value of \( Y \) is known. It is defined as:

$$
H(X|Y) = - \sum_{x, y} p(x, y) \log p(x|y)
$$

Substituting the definition of conditional probability \( p(x|y) \):

$$
H(X|Y) = - \sum_{x, y} p(x, y) \log \left( \frac{p(x, y)}{p(y)} \right)
$$

Using the properties of logarithms, we can separate the terms inside the logarithm:

$$
H(X|Y) = - \sum_{x, y} p(x, y) \left[ \log p(x, y) - \log p(y) \right]
$$

Distributing \( p(x, y) \) and summing over all \( x \) and \( y \):

$$
H(X|Y) = - \sum_{x, y} p(x, y) \log p(x, y) + \sum_{x, y} p(x, y) \log p(y)
$$

Notice that the second term simplifies because $$ \sum_{x} p(x, y) = p(y) $$

$$
H(X|Y) = H(X, Y) - H(Y)
$$

Thus, the conditional entropy \( H(X|Y) \) is the difference between the joint entropy \( H(X, Y) \) and the entropy \( H(Y) \). This makes sense intuitively, as it measures the remaining uncertainty in X after taking into account the information provided by Y.

The definition involves the ratio \( p(x, y) / p(y) \) because it represents the conditional probability ( p(x|y) ), which is crucial for understanding how the knowledge of ( Y ) affects the uncertainty of ( X ).
